#### <font color="#03997A"> Diplomado en Ciencia de datos UNAM </font>

##### <font color="#03997A"> Dr: Alejandro Pimentel  </font>

##### <font color="#03997A"> Modulo 8 Introducción al Deep Learning </font>

##### <font color="#03997A"> Tema 5 Encoder-Decoer  Alumno: Ibarra Ramírez Sergio </font>

In [1]:
import tensorflow as tf

In [2]:
max_oracion = 300
num_palabras = 5000
emb_dim = 200

#### De que tamanio va ser el vector con que se representarña la oración total
h_dim = 600

Las palabras vectorizadas, vana ser las entradas, pasadas por un embeding (Es como se hacía en el secuencial en donde se definnia como primera capa un embeing donde se especificaba su tamanio)

### Encoder 

In [3]:
entradas_codificador = tf.keras.layers.Input(shape=(None,))
embeddings = tf.keras.layers.Embedding(num_palabras,emb_dim)
palabras_vectorizadas = embeddings(entradas_codificador)
palabras_vectorizadas

<KerasTensor: shape=(None, None, 200) dtype=float32 (created by layer 'embedding')>

Se va ausar la arquitectura GRU como encodficiador

In [4]:
codificador_gru = tf.keras.layers.GRU(h_dim,recurrent_dropout=0.5,return_state=True)
salida_cod, estado_cod = codificador_gru(palabras_vectorizadas)
salida_cod


<KerasTensor: shape=(None, 600) dtype=float32 (created by layer 'gru')>

In [5]:
modelo_codificador = tf.keras.models.Model(entradas_codificador,estado_cod)
modelo_codificador

### Decoder 

Y las entradas del decodificador será obviamente las salidas del codificador 

In [6]:
entradas_decodificador = tf.keras.layers.Input(shape=salida_cod.shape)
entradas_decodificador


<KerasTensor: shape=(None, None, 600) dtype=float32 (created by layer 'input_2')>

Po tro lado return_sequences retorna una matriz como secuencia de palabras de salida

In [7]:
decodificador_gru = tf.keras.layers.GRU(h_dim,recurrent_dropout=0.5, return_sequences=True)
decodificador_gru

In [8]:

entrada_estado_decod = tf.keras.layers.Input(shape=(h_dim,))
salidas_decod = decodificador_gru(entradas_decodificador,initial_state=entrada_estado_decod)
salidas_decod

<KerasTensor: shape=(None, None, 600) dtype=float32 (created by layer 'gru_1')>

In [9]:
decodificador_densa = tf.keras.layers.Dense(num_palabras, activation='softmax')
decodificador_densa 


Y la ultima capa despues de las primeras dos RNN (En este caso GRU) será una Dense de tipo SOFTMAX, en donde la salida de la GRU Decoder se le pasa al a densa de SOFTMAX para predecir cada caso la palabras más probable siguiente 

In [10]:
salidas_decod = decodificador_densa(salidas_decod)
modelo_decodificador = tf.keras.models.Model([entrada_estado_decod,entradas_decodificador],salidas_decod)
modelo_decodificador